In [5]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods 
import make_question as question_maker
import sequence_handler
import corpus_handling_methods
import pandas as pd
import ranking
import re
from gensim.models import Word2Vec
import numpy as np
import MeCab
mt = MeCab.Tagger('')
mt.parse('')

gensim_model = Word2Vec.load('/home/iftekhar/myworkplace/AI-system/2VECs_models/word2vec/combined.model')

In [6]:
dataset = pd.read_csv("../../../Helpers/Title_link_merged_corpus.csv")
dataset = dataset.iloc[:,2:]
dataset = dataset.rename(columns={"Article": "Data"})
dataset.Data = dataset.Data.apply(lambda x: corpus_handling_methods.clean_text(x))
dataset.head()

,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [7]:
split_corpus = corpus_handling_methods.corpus_split(dataset, 8)
split_corpus.head()

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,設定 順序 MobiControl 設定 順序 行い 端末 OS,0
2,対象 オンプレミス MobiControl サーバ のみ 必要 な 設定,0
3,事項 iOS 端末 macOS コンピュータ 必須 接続 プロファイル,0
4,作成 ADDS Active Directory Domain Service サーバ 間,0


In [8]:
perpage_dataset = corpus_handling_methods.corpus_per_page(split_corpus)
perpage_dataset.head() 

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,4


In [29]:
sample_size = 100
questions_samples = question_maker.question_dataframe_generator_1000(split_corpus, sample_size)
questions_samples.head()

,Question,PageID
0,インストール べき に関する 情報 送り 送る べき どうでしたか,430
1,れ いる アプリ アイコ 表示 MobiControl アイコ どうでしたか,252
2,削除 時間 WIPE 自己 実行 工場 出荷 始める方法,152
3,開発 社内 Windows Phone 事前 AET Application 入れられますか,92
4,Plus Android 問わ ず コンソール 当該 プロファイル 何が,243


In [30]:
sample_count = 0
sum_score = 0
container = []

for index, col in questions_samples.iterrows():
    #print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    #ranks = ranking.crude_ranks(perpage_sequence_match, query, vec)
    #filtered_ranks = ranking.filtering_ranks(ranks, perpage_sequence_match, query, vec)
    
    ranks = word2vec_ranks(perpage_sequence_match, query)
    
    page_answers = []
    prediction_scores = []
    for ids, score in ranks:
        page_answers.append(ids)
        prediction_scores.append(score)
    MRR_score = ranking.mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score += MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
    sample_count += 1
    
result = pd.DataFrame(container, columns=['score', 'actual_answer',
'page_answers', 'prediction_scores', 'query'])
result.to_csv('seq_matcher_TFIDF_Vectorizer_performance.csv')
score = sum_score/sample_count
score

/home/iftekhar/myworkplace/environments/experiment_env/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


0.7466666666666666

In [10]:
sample_count = 0
sum_score = 0
container = []

for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    break

CE 後継 ある Windows Embedded プラットフォーム 電話機 なにで 115


In [14]:
def get_vector(text):
    sum_vec = np.zeros(200)
    word_count = 0
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
            try: 
                temp = gensim_model.wv[node.surface]
            except KeyError:
                temp = 0
            sum_vec += temp
            word_count += 1
        node = node.next
    return sum_vec / word_count

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def word2vec_ranks(perpage_sequence_match, query):
    X = get_vector(query)
    rank = []
    ids_list = []
    for ids, items in perpage_sequence_match:
        for sentences in items:
            Y = get_vector(sentences)
            ids_list.append(ids)
            rank.append(cos_sim(X, Y))
    return sorted(zip(ids_list, rank), key=lambda l:l[1], reverse=True)[:3]